###Assignment 6
[3 points] Delelop language model, which generates texts from wikipedia.
Use WikiText-2 dataset, also available in torchtext.datasets.
Use sentencepiece or tokenizers library for text tokenization. Pay attention to vocab size, probably subword tokens are better.
Your model should be autogressive RNN.


[1 point] Plot train and validation loss depending on the number of iterations of gradient decent.

[1 point] Try to use together (sentencepiece or tokenizers), torchtext.datasets, and torchtext.data.BPTTIterator

Text generation should be terminated when either max length is reached or terminal symbol is generated.

Explore several inference techniques:

[1 point] Argmax

[1 point] Beamsearch

[1 point] Sampling from probabilty distribution with temperature

[1 point] Nucleus sampling

[1 point] Top-k sampling

For every method you should provide implemented code and generated examples. Each example must contain at least 10 words (not subword tokens).
Readings: https://arxiv.org/abs/1904.09751


In [1]:
import pandas as pd
import numpy as np
from sklearn.externals import joblib
import nltk
import gensim
import spacy
from tqdm import tqdm_notebook

from sklearn import metrics

import torch as tt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchtext.data import Field, LabelField, BucketIterator, ReversibleField



SEED = 42
np.random.seed(SEED)

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [0]:
from torchtext.datasets import WikiText2

In [3]:
WikiText2.download('.')

downloading wikitext-2-v1.zip


wikitext-2-v1.zip: 100%|██████████| 4.48M/4.48M [00:01<00:00, 3.01MB/s]


extracting


'./wikitext-2/wikitext-2'

In [4]:
!pip install tokenizers

     |████████████████████████████████| 3.7MB 2.6MB/s 


In [0]:
import tokenizers

In [0]:
import spacy

spacy_en = spacy.load('en')

In [0]:
def tokenizer(text): # create a tokenizer function
    return [tok.lemma_ for tok in spacy_en.tokenizer(text) if tok.text.isalpha()]

In [9]:
tokenizer("Hello! It is me")

['Hello', 'It', 'be', 'me']

In [10]:
from spacy.lang.en import English
nlp = English()
sbd = nlp.create_pipe('sentencizer')
nlp.add_pipe(sbd)

text="Please read the analysis. (You'll be amazed.)"
doc = nlp(text)

sents_list = []
for sent in doc.sents:
   sents_list.append(sent.text)

print(sents_list)
print([token.text for token in doc])

['Please read the analysis. (', "You'll be amazed.)"]
['Please', 'read', 'the', 'analysis', '.', '(', 'You', "'ll", 'be', 'amazed', '.', ')']


In [0]:
def sentence_tokenizer(text):
    nlp.max_length = len(text) + 1
    return [i for i in nlp(text).sents]

In [0]:
directory = './wikitext-2/wikitext-2'

In [0]:
from os import path

with open(path.join(directory, 'wiki.valid.tokens'), 'r') as f:
    val = f.read()
with open(path.join(directory, 'wiki.train.tokens'), 'r') as f:
    train = f.read()
with open(path.join(directory, 'wiki.test.tokens'), 'r') as f:
    test = f.read()


In [16]:
print(train[:500])

 
 = Valkyria Chronicles III = 
 
 Senjō no Valkyria 3 : <unk> Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . <unk> the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs paralle


In [0]:
raw = [val, train, test]
prepared = []
for text in raw:
    sentences = sentence_tokenizer(text)
    prepared_text = ['<start> {} <end>'.format(sent) for sent in sentences if sent]
    prepared.append(prepared_text)

In [21]:
prepared[0][:5]

['<start>  \n = Homarus gammarus = \n \n Homarus gammarus , known as the European lobster or common lobster , is a species of <unk> lobster from the eastern Atlantic Ocean , Mediterranean Sea and parts of the Black Sea . <end>',
 '<start> It is closely related to the American lobster , H. americanus . <end>',
 '<start> It may grow to a length of 60 cm ( 24 in ) and a mass of 6 kilograms ( 13 lb ) , and bears a conspicuous pair of claws . <end>',
 '<start> In life , the lobsters are blue , only becoming " lobster red " on cooking . <end>',
 '<start> Mating occurs in the summer , producing eggs which are carried by the females for up to a year before hatching into <unk> larvae . <end>']